## **Assignment 4 - Knowledge Graph Quality**

The subject of this assignment is the development and implementation of a quality assessment on the DBPedia knowledge graph.

The assignment consists of 2 tasks, where the first task asks to implement a term ambiguity detector that identifies ambiguous class and property names using an LLM approach and also a dictionary-based approach, while the second task asks to quality check how well or not DBPedia models abstract concepts by running an experiment considering ESCO skill entities.

In the next two chapters we will discuss the steps and design decisions taken in both tasks to achieve the desired results.

### **Task 1 - Term Ambiguity Detector**

In this task, I was tasked with developing a term ambiguity detector to evaluate the ambiguity of class and property names in the DBPedia graph, using two different approaches. For the first approach, I had to implemement a llm-based solution and chose the ChatGPT's 'gpt-4o-mini' model. For the second approach, I relied on a dictionary-based method, using WordNet for ambiguity evaluation.

Here are the steps I followed:

1. Develop a function to retrieve the names of the classes by executing a SPARQL query.
2. Develop a function to retrieve the names of the properties using a SPARQL query.
2. Develop a function that prompts the 'gpt-4o-mini' model to determine whether a term (such as a class or property name) is ambiguous, and provide a justification (LLM-based approach).
4. Develop a function that integrates with the WordNet dictionary to identify all possible senses of a term (such as a class or property name) and assess whether it is ambiguous (dictionary-based approach).

Let's explain each step in more detail to understand the development process:
    
**1. Gather class names**

Here i wrote a SPARQL query which finds and retrieves the distinct class names. For the purposes of the task, i put a limit on the number of retrieved names (i kept only 50), so that the results of both approaches (llm vs dictionary) can be more easily viewed and analyzed. Of course, if we want we can remove this limit and get the results for all data.

<u>Query:</u>

    SELECT 
        DISTINCT (STR(?name) AS ?className)
    WHERE {
        ?classId rdf:type owl:Class .
        OPTIONAL {
            ?classId rdfs:label ?name .
            FILTER(LANG(?name) = "en")
        }
    }
    LIMIT 50

**2. Gather property names**

Here i wrote a SPARQL query which finds and retrieves the distinct property names. For the purposes of the task, i put a limit on the number of retrieved names (i kept only 50), so that the results of both approaches (llm vs dictionary) can be more easily viewed and analyzed. Of course, if we want we can remove this limit and get the results for all data.

<u>Query:</u>

    SELECT 
        DISTINCT (STR(?name) AS ?propertyName)
    WHERE {
        ?propertyId rdf:type rdf:Property .
        OPTIONAL {
            ?propertyId rdfs:label ?name .
            FILTER(LANG(?name) = "en")
        }
    }
    LIMIT 50

**3. Prompt ChatGPT (llm-based approached)**

Here I wrote a proper prompt for communicating with ChatGPT and let it decide whether a term is ambiguous or not.

<u>Prompt:</u>

    You are an system that helps assess the ambiguity of a term. Your job is to determine whether a given term is ambiguous or not.

    Some cases to help you understand when a term is ambiguous or not:
        - A term is ambiguous when it has multiple meanings or it is uncertain (e.g., "bat" can refer to an animal or a piece of sports equipment.).
        - A term is ambiguous when its meaning changes depending on the context in which it is used (e.g., "scale" can mean a musical scale in music, a device for weighing in
          measurement, or even a pattern on fish skin in biology.).
        - A term is ambiguous when it has different meanings in different fields or industries (e.g., In sports, "run" refers to moving quickly on foot and in programming, "run" 
          means to execute a program or script.). 

    Important Instructions:
        - Return the word "Yes" if the term is ambiguous or the word "No" if it is not and also a sentence explaining why it is ambiguous or not and always include the term to 
          know what it refers to.

At the beginning I give him the concept of the work he has to do and then I give him some cases where a term can be considered ambiguous or not, including some examples (a combination of zero-shot and few-shot learning) and finally give him instructions on the format of how the results should be.


**4. Integrate with Wordnet (dictionary-based approach)**

Here I take advantage of the Wordnet dictionary to see how many meanings each term (class or property name) has and determine whether it is ambiguous or not. 

I observed that some names consist of a single word, while others are made up of multiple words, which complicates assessing the ambiguity of each name. 

When the name is a single word, I can simply check it against WordNet to see how many synsets it has and if there is more than one, the name is considered ambiguous due to its multiple potential interpretations. However, for multi-word names, there are two possibilities, either the name is recognized in the dictionary, making it easier to assess its ambiguity level (if there is more than one synset then it is ambiguous otherwise it is not) or it isn't listed as a complete phrase, making it difficult to determine ambiguity. In such cases, the best approach is to analyze the synsets of each individual word in the dictionary and then infer the overall ambiguity of the name based on the ambiguity level of its components (if each component has only one synset then the multi-word name is not ambiguous otherwise it is), which is not always a good approach, because there are many cases where individual words are ambiguous but their combination is not.


So, based on the above, I took the following steps:

1. Identify the synsets associated with the entire name.
2. Identify the synsets of each individual word within the name.
3. Compare 
    - If the name is a single word assess its ambiguity based on its synsets.
        - Only one -> Non-Ambiguous
        - Zero or More than one -> Ambiguous
    - If the name is a multi-word and itself has one or more synsets, assess its ambiguity based on those synsets. 
        - Only one -> Non-Ambiguous
        - More than one -> Ambiguous
    - If the name is a multi-word has no standalone synset, use the synsets of its components to infer the overall ambiguity of the name.
        - All components with one synset -> Non-Ambiguous
        - At least one component with zero or more than one synset -> Ambiguous

<br>

#### **Effectiveness and Efficiency**

**Dictionary-Based Approach**

WordNet, like any other dictionary, has extensive coverage of common English words, making this approach effective especially for terms that are widely used. However, a dictionary is very likely to not contain specific and domain-specific terms reducing the coverage. Also, by using a dictionary in cases where we have a term with more than one word we may not always get the right results on whether it is ambiguous or not. This happens because the term may not be in the dictionary and in order to assess whether it is ambiguous or not we look at the ambiguities of the components of the term, which does not always lead to safe conclusions. It is possible each component or few of them to be ambiguous because they may have many meanings but the combination of these components give a term that has a specific meaning and is not ambiguous. Finally, dictionary lookups are generally fast and do not require much computational power

**LLM-Based Approach**

LLM models are trained on a huge corpus of texts and can capture nuances of concepts, even for uncommon or specific terms. This makes the LLM approach particularly effective for identifying ambiguous terms in specialized domains. LLM models are well suited for handling multi-word terms (in contrast to the dictionary-based approach), as they process phrases as coherent units, taking into account both individual words and their combined meaning. Also, in the LLM approach we have the ability to adapt the behavior and decision making of the system by providing appropriate instructions and examples via promt making the model more able to understand and correctly determine whether it is ambiguous or not. Finally, LLM calls are relatively slow and computationally expensive, especially for large datasets.


Based on the 'quality_detector' notebook which implements this task, the number of class names where both approaches (llm-based and dictionary-based) agree and return the same ambiguity type is 36 while the agreement for the property names is 22.

Let's make some comments to analyze the behavior of both approaches:

- When we have a specialized or unusual term, the dictionary-approach has difficulty finding its meaning and therefore cannot correctly determine whether it is ambiguous or not. On the other hand, llm-approach is more able to determine correctly as it has been trained in a huge text dataset including very specific terms.
  
    <u>Example</u>
    |Name|ChatGPT|Wordnet|
    |--------|---------|------|
    |Gnetophytes|Non-Ambiguous|Ambiguous|   

    <u>Explain:</u>
    llm-approach correctly says that is not ambiguous while dictionary-approach fails to assess the correct ambiguity since it could not find the senses of the word                   

- When we have a name that is a generic term with a known meaning, the dictionary approach is able to find its meanings and decide on the ambiguity, as well as an LLM system.

    <u>Example</u>
    |Name|ChatGPT|Wordnet|
    |--------|---------|------|
    |Activity|Ambiguous|Ambiguous|     

    <u>Explain:</u>
    llm-approach and dictionary-approach correctly say that is ambiguous    

- When we have a term consisting of many words, the dictionary approach is more likely to fail to understand the term and decide whether it is ambiguous or not. This probability is very high when the phrase is not frequently used and therefore not logically included in the dictionary, and is minimized when the phrase is very common and much used and therefore more likely to be in the dictionary. Unlike the llm approach where regardless of whether the phrase is specific or not the model is able to estimate the level of ambiguity as it is trained on a huge amount of data.

    <u>Example (Common term)</u>
    |Name|ChatGPT|Wordnet|
    |--------|---------|------|
    |chemical substance|Non-Ambiguous|Non-Ambiguous|        

    <u>Explain:</u>
    llm-approach and dictionary-approach correctly say that is not ambiguous

    <br>

    <u>Example (Specific/not common term)</u>
    |Name|ChatGPT|Wordnet|
    |--------|---------|------|
    |Wikipage page ID|Non-Ambiguous|Ambiguous|      

    <u>Explain:</u>
    llm-approach correctly says that is not ambiguous while dictionary-approach fails to assess  the correct ambiguity

- When we have a term consisting of many words, if it is not included in the dictionary as it is, then one approach to estimating the ambiguity, as I said before, is to find the ambiguity of the individual words in the term and then determine the overall ambiguity based on them, which most of the time gives bad results but we do not have other option.

    <u>Example</u>
    |Name|ChatGPT|Wordnet|
    |--------|---------|------|
    |non-profit organisation|Non-Ambiguous|Ambiguous|      
    
    <u>Explain:</u>
    llm-approach correctly says that is not ambiguous while dictionary-approach fails to assess  the correct ambiguity

**Conclusion:**

The dictionary-based approach is ideal for quickly identifying ambiguous terms when dealing with general-purpose or single-word terms, offering a fast and cost-effective solution. However, it struggles with multi-word or domain-specific terms. In contrast, the LLM-based approach excels at handling complex, nuanced ambiguities, especially with multi-word or context-dependent terms, but comes with higher computational costs and processing time. Therefore, the dictionary approach is suitable for large datasets with limited resources, while the LLM is preferable when accuracy and detailed interpretation are prioritized. A hybrid approach can balance efficiency and precision by combining both methods.


**Just to mention an idea I had, when we have several meanings for a term through a dictionary, we could check how close they are semantically, so that if they are too close we could consider them as one and so the term would not be ambiguous. This can be applied to terms consisting of both one word and many words. I have not had time to test and apply this concept.

### **Task 2**

In this task, I had to find DBPedia entities that are equivalent to some ESCO entities given in a CSV file and their associated classes. Then the next step was to use an LLM to evaluate all entity-class pairs as to how well and accurately each class describes the corresponding entity.

Let's start with the first part which is about entity mapping between ESCO and DBPedia.

To find the mapped entities I decided to use the ESCO entity labels and run SPARQL queries based on them. To do this, I encountered some problems which are listed below.

- **preferredLabel or altLabels**

My first thought was to use both columns from the CSV file, as the first one provides the main label of the entity and the second one provides all the alternative labels associated with it. However, running all the queries with all of them was very time consuming and not efficient. I used several ways to do it, such as one by one, using the VALUES clause to include all the tags (main and alternative) for each entity and checking if it exists based on them. Both run very slowly and i could not get the desired results.

<u>one by one:</u>

    ?entity rdfs:label "label"@en
    
<u>VALUES clause:</u>

    VALUES ?label {"label1"@en "label2"@en "label3"@en}
    ?entity rdfs:label ?label

- **rdfs:label - skos:altLabel - dbp:alias - foaf:name**

My next thought was what RDF properties I should use to properly check if a label exists in the DBPedia. All of these (rdfs:label - skos:altLabel - dbp:alias - foaf:name) in DBPedia are used to provide human-readable alternative names and aliases of entities. Certainly, the main one is 'rdfs:label' which contains the main human-readable name/label of the entity. I tried to use all of these in my query at the beginning, but again as before it was very time consuming and not efficient.

- **SPARQL - Case Sensitivity**

I noticed that in the CSV file most labels include words whose first character is lowercase unlike DBPedia where most entity labels consist of words where the first character is uppercase (especially for the first word). I tried to make the queries and the 'string' comparisons between the labels case insensitive through many ways such as:

    ?entity rdfs:label ?label
     FILTER (LCASE(?label) = "label")

    ?entity rdfs:label ?label
    FILTER (REGEX(?label, "^{label}$", "i"))

but again the execution was very low.


Based on the above, I took the following decisions:

- To reduce the query runtime I decided to use only the labels included in the "preferredLabel" column, as they are the main human-readable names.
- To reduce the query runtime I decided to use only the 'rdfs:label' property which as I noticed is the most common way of expressing entity aliases in DBPEdia.
- Since I could not find a dynamic way to disregard whether words are upper or lower case through the SPARQL query, I pre-processed the CSV file and for each label from the preferredLabel column I got two additional variants. One in which all words in the label have an uppercase first character and another in which only the first word in the label has an uppercase first character. I did this after researching a lot into the structure of the main entity labels in DBPedia and also to match more relevant entities.

The query I used to run it multiple times to match entities is:

    SELECT DISTINCT (STR(?classLabel) AS ?className) WHERE {{
        ?entity rdfs:label "{label}"@en .
        ?entity rdf:type ?class .
        ?class rdfs:label ?classLabel .
        FILTER (LANG(?classLabel) = "en")
        FILTER (STRSTARTS(STR(?entity), "http://dbpedia.org/resource/"))
        FILTER (STRSTARTS(STR(?class), "http://dbpedia.org/ontology/"))
        FILTER NOT EXISTS {{ ?superclass rdfs:subClassOf ?class . ?entity rdf:type ?superclass . }}
    }}

In the query I'm trying to find an entity that has a specific English tag '"{label}"@en' and return all associated classes except those that are parent classes of an existing one, so that only the direct relationship between the entity and the classes is preserved and their parent class is not taken. I define the point where entities and classes are referenced in DBPedia as 'http://dbpedia.org/resource/' and 'http://dbpedia.org/ontology/' respectively. 


Now, to assess how well and accurate each class describe an entity i used the ChatGPT's 'gpt-4o-mini' model. I used the following prompt:

    
    system_content = f"""
    You are an system that judge if a entity-class pair in a knowledge graph is accurate or not. Your job is to determine if the given class accurately describe the given entity.\n
    Please return "Yes" or "No"
    """

    user_content = f"""
    Entity: "{entity}"\n\
    Proposed Class: "{enity_class}"
    Question: Does the proposed class "{enity_class}" accurately describe the entity "{entity}"?
    """

In the system role, I explained what the system should do and what the results should be and through the user role, I provided the entity and class of the pair to judge the model whether it is accurate or not.

**Findings**

Based on the steps I followed, 590 ESCO entities were identified with DBPedia entities. Some entities had more than one associated class and the entity-class pairs deemed accurate or inappropriate by the LLM are as follows:

|Accurate|Inaccurate|
|--------|---------|
|110|668|Ambiguous| 

The imprecise relationships between most of the entity-class pairs arose because most of the classes are too abstract and vague, while the entities that fall under them are more specific and are not accurately described by the corresponding classes. The classes with the most errors are listed below, which means that they do not adequately explain the entities falling under them. Semantically they are not easily linked to the entities they contain.

|Class|Mistakes|
|--------|---------|
|person function|145|
|music genre|115|
|book|71|
|organisation|71|
|university|30|
|work|24|

On the other hand, the classes that were found to be correct most of the time and accurately described the entities falling under them are listed below. These classes are more specific and easier to understand what they represent. They have a greater semantic relationship with their entities and thus explain them better.

|Class|Correct|
|--------|---------|
|software|45|
|programming language|16|
|medical specialty|13|
|language|12|

<br>

Thus, DBPedia does not always model abstract concepts correctly.